# 15-03 양방향 LSTM과 어텐션 메커니즘 (BiLSTM with Attention Mechanism)

## 1. IMDB 리뷰 데이터 전처리하기

In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocab_size)

# 최대 단어 개수를 10,000으로 제한, 훈련 데이터와 테스트 데이터 받아온다.
# 훈련 데이터와 레이블이 각각 X_train, y_train에 테스트 데이터 저장
# 테스트 데이터와 이에 대한 레이블이 X_text, y_test에 저장
# IMDB 리뷰 데이터는 이미 정수 인코딩이 되어있어, 남은 전처리는 패딩 뿐이다

In [4]:
print('리뷰의 최대 길이 : {}'.format(max(len(l) for l in X_train)))
print('리뷰의 평균 길이 : {}'.format(sum(map(len, X_train))/len(X_train)))

리뷰의 최대 길이 : 2494
리뷰의 평균 길이 : 238.71364


In [5]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

# 길이 500으로 설정하고 패딩.

## 2. 바다나우 어텐션(Bahdanau Attention)

- 어텐션 함수란 주어진 query와 모든 key에 대해서 유사도를 측정하는 함수를 말한다.

In [6]:
import tensorflow as tf

class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): # 단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

## 3. 양방향 LSTM + 어텐션 메커니즘(BiLSTM with Attention Mechanism)

In [7]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

- 모델 설계 시작

In [9]:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, mask_zero = True)(sequence_input)
# 입력층과 임베딩층을 설계
# 10,000개의 단어들을 128차원의 벡터로 임베딩하도록 설계

In [10]:
lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences = True))(embedded_sequences)
# 양방향 LSTM 설계.
# 여기서는 두 층 사용
# 우선 첫번째 층 만든다, 두번째 층을 위해 쌓을 예정이므로 return_sequences를 True로 해줘야 한다

In [11]:
# 두번째 층 설계, 상태 리턴 받아야해서 return_state를 True로
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
  (LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

In [12]:
# 각 상태의 크기(shape) 출력
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

(None, 500, 128) (None, 64) (None, 64) (None, 64) (None, 64)


- 순방향 LSTM의 은닉 상태와 셀상태를 forward_h, forward_c에 저장하고 역방향의 LSTM의 은닉 상태와 셀 상태를 backward_h, backward_c에 저장.
- 은닉 상태나 셀 상태의 경우에는 128차원을 가진다, lstm의 경우는 500x128의 크기를 가진다.
- forward 방향과 backward 방향이 연결된 hidden state 벡터가 모든 시점에 대해서 존재함을 의미한다.
- 양방향 LSTM을 사용할 경우, 순방향 LSTM과 역방향 LSTM 각각 은닉 상태와 셀 상태를 가진다. 양방향 LSTM의 은닉 상태와 셀 상태를 사용하려면 두 방향의 LSTM의 상태들을 연결(concatenate)해주면 된다.

In [13]:
state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

In [14]:
# 어텐션 메커니즘에서는 은닉상태 사용. 이를 입력으로 컨텍스트 벡터(context vector) 얻는다
attention = BahdanauAttention(64) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

- 컨텍스트 벡터를 밀집층(dense layer)에 통과 시키고, 이진 분류이므로 최대 출력층에 1개의 뉴런을 배치, 활성화 함수로 시그모이드 함수 사용

In [15]:
dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = Model(inputs=sequence_input, outputs=output)

In [16]:
# 옵티마이저로 아담 옵티마이저 사용, 모델 컴파일
# 시그모이드 함수를 사용하므로, 손실 함수로 binary_crossentropy 사용
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# 모델 훈련
history = model.fit(X_train, y_train, epochs = 3, batch_size = 256, validation_data=(X_test, y_test), verbose=1)
# 검증 데이터로 테스트 데이터 사용해서 에포크가 끝날 때마다 테스트 데이터에 대한 정확도 출력

Epoch 1/3
98/98 [==============================] - 1326s 13s/step - loss: 0.4959 - accuracy: 0.7610 - val_loss: 0.3157 - val_accuracy: 0.8628
Epoch 2/3
98/98 [==============================] - 1293s 13s/step - loss: 0.2584 - accuracy: 0.9070 - val_loss: 0.2934 - val_accuracy: 0.8800
Epoch 3/3
98/98 [==============================] - 1289s 13s/step - loss: 0.1988 - accuracy: 0.9312 - val_loss: 0.2947 - val_accuracy: 0.8773


In [18]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 343s 439ms/step - loss: 0.2947 - accuracy: 0.8773

 테스트 정확도: 0.8773
